In [1]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datetime import datetime
from tqdm import tqdm
from glob import glob

In [2]:
CUDA_DEV = 0
NUM_TAGS = 256

In [3]:
data_dir = '/kaggle/input/yandex-cup-2023-recsys'
emb_dir = 'track_embeddings'

df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [4]:
track_idx2embeds = {}
for fn in tqdm(glob(os.path.join(data_dir, emb_dir) + '/*')):
    filename = os.path.basename(fn)
    track_idx = int(os.path.splitext(filename)[0])
    embeds = np.load(fn)
    track_idx2embeds[track_idx] = embeds

len(track_idx2embeds)

100%|██████████| 76714/76714 [09:38<00:00, 132.70it/s]


76714

In [5]:
class TaggingDataset(Dataset):
    def __init__(self, df, testing=False):
        self.df = df
        self.testing = testing
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        track_idx = row.track
        embeds = track_idx2embeds[track_idx]
        if self.testing:
            return track_idx, embeds
        tags = [int(x) for x in row.tags.split(',')]
        target = np.zeros(NUM_TAGS)
        target[tags] = 1
        return track_idx, embeds, target

In [6]:
train_dataset = TaggingDataset(df_train)
test_dataset = TaggingDataset(df_test, True)

In [7]:
class Network(nn.Module):
    def __init__(
        self,
        num_classes = NUM_TAGS,
        input_dim = 768,
        hidden_dim = 512
    ):
        super().__init__()
        self.num_classes = num_classes
        self.bn = nn.LayerNorm(hidden_dim)
        self.projector =  nn.Linear(input_dim, hidden_dim)
        self.lin = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim)
        )
        self.fc = nn.Linear(hidden_dim, num_classes)
        

    def forward(self, embeds):
        x = [self.projector(x) for x in embeds]
        x = [v.mean(0).unsqueeze(0) for v in x]
        x = self.bn(torch.cat(x, dim = 0))
        x = self.lin(x)
        outs = self.fc(x)
        return outs


In [8]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = None
    alpha = 0.8
    for iteration,data in enumerate(loader):
        optimizer.zero_grad()
        track_idxs, embeds, target = data
        embeds = [x.to(CUDA_DEV) for x in embeds]
        target = target.to(CUDA_DEV)
        pred_logits = model(embeds)
        pred_probs = torch.sigmoid(pred_logits)
        ce_loss = criterion(pred_logits, target)
        ce_loss.backward()
        optimizer.step()
        
        if running_loss is None:
            running_loss = ce_loss.item()
        else:
            running_loss = alpha * running_loss + (1 - alpha) * ce_loss.item()
        if iteration % 100 == 0:
            print('   {} batch {} loss {}'.format(
                datetime.now(), iteration + 1, running_loss
            ))

In [9]:
def predict(model, loader):
    model.eval()
    track_idxs = []
    predictions = []
    with torch.no_grad():
        for data in loader:
            track_idx, embeds = data
            embeds = [x.to(CUDA_DEV) for x in embeds]
            pred_logits = model(embeds)
            pred_probs = torch.sigmoid(pred_logits)
            predictions.append(pred_probs.cpu().numpy())
            track_idxs.append(track_idx.numpy())
    predictions = np.vstack(predictions)
    track_idxs = np.vstack(track_idxs).ravel()
    return track_idxs, predictions

In [10]:
def collate_fn(b):
    track_idxs = torch.from_numpy(np.vstack([x[0] for x in b]))
    embeds = [torch.from_numpy(x[1]) for x in b]
    targets = np.vstack([x[2] for x in b])
    targets = torch.from_numpy(targets)
    return track_idxs, embeds, targets

def collate_fn_test(b):
    track_idxs = torch.from_numpy(np.vstack([x[0] for x in b]))
    embeds = [torch.from_numpy(x[1]) for x in b]
    return track_idxs, embeds

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, collate_fn=collate_fn_test)

In [12]:
model = Network()
criterion = nn.BCEWithLogitsLoss()

epochs = 100
model = model.to(CUDA_DEV)
criterion = criterion.to(CUDA_DEV)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in tqdm(range(epochs)):
    train_epoch(model, train_dataloader, criterion, optimizer)

  0%|          | 0/100 [00:00<?, ?it/s]

   2023-10-25 18:20:00.689212 batch 1 loss 0.7380401492561646
   2023-10-25 18:20:04.804588 batch 101 loss 0.08221160314581685
   2023-10-25 18:20:08.871037 batch 201 loss 0.06677076556559228
   2023-10-25 18:20:12.971962 batch 301 loss 0.06113852584081181


  1%|          | 1/100 [00:18<29:50, 18.08s/it]

   2023-10-25 18:20:16.993327 batch 1 loss 0.06180681880682215
   2023-10-25 18:20:21.088439 batch 101 loss 0.05847679932389772
   2023-10-25 18:20:25.235757 batch 201 loss 0.05609370486928812
   2023-10-25 18:20:29.351700 batch 301 loss 0.05563216616552631


  2%|▏         | 2/100 [00:34<27:55, 17.10s/it]

...

 98%|█████████▊| 98/100 [26:35<00:32, 16.28s/it]

   2023-10-25 18:46:34.391515 batch 1 loss 0.030857259624481298
   2023-10-25 18:46:38.694564 batch 101 loss 0.032198381246665096
   2023-10-25 18:46:42.859210 batch 201 loss 0.03327277193791083
   2023-10-25 18:46:46.930307 batch 301 loss 0.033411353198106494


 99%|█████████▉| 99/100 [26:52<00:16, 16.40s/it]

   2023-10-25 18:46:51.065061 batch 1 loss 0.030924108633544268
   2023-10-25 18:46:55.098419 batch 101 loss 0.03223530343964418
   2023-10-25 18:46:59.320140 batch 201 loss 0.03221924849145485
   2023-10-25 18:47:03.806772 batch 301 loss 0.03275228829756491


100%|██████████| 100/100 [27:09<00:00, 16.29s/it]


In [13]:
%%time
track_idxs, predictions = predict(model, test_dataloader)

CPU times: user 4.56 s, sys: 19.9 ms, total: 4.58 s
Wall time: 4.58 s


In [14]:
predictions_df = pd.DataFrame([
    {'track': track, 'prediction': ','.join([str(p) for p in probs])}
    for track, probs in zip(track_idxs, predictions)
])

In [15]:
submit_file = 'submit_batch-128_epochs-100.csv'
predictions_df.to_csv(submit_file, index=False)

In [16]:
from IPython.display import FileLink

FileLink(submit_file)

/kaggle/working/submit_batch-128_epochs-100.csv